**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/as-with).**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. 

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-oxabyqo0
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-oxabyqo0
  Resolved https://github.com/Kaggle/learntools.git to commit 940f894c224b3737a597a80e9a792face28c6562
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268968 sha256=6a319bc686abc80c81a8e171f5faa703394ed0844c45f3c46379a890e260b44c
  Stored in directory: /tmp/pip-ephem-wheel-cache-zqc5p4bi/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


You'll work with a dataset about taxi trips in the city of Chicago. Run the cell below to fetch the `chicago_taxi_trips` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

### 1) Find the data
Before you can access the data, you need to find the table name with the data.

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [12]:
# Your code here to find the table name
tables = list(client.list_tables(dataset))
for t in tables:
    print(t.table_id)


taxi_trips


In [13]:
# Write the table name as a string below
table_name = 'taxi_trips'

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [14]:
#q_1.solution()

### 2) Peek at the data

Use the next code cell to peek at the top few rows of the data. Inspect the data and see if any issues with data quality are immediately obvious. 

In [19]:
# Your code here
table_ref = dataset_ref.table('taxi_trips')
table = client.get_table(table_ref)
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,d0dd5f20073d6174a456961088231dc46006a5c0,efce437d4bc2a79b12ac131e37789da8fe5e53f37d12c2...,2015-10-17 19:15:00+00:00,2015-10-17 19:30:00+00:00,780,2.2,1.703107e+10,1.703106e+10,7,6,...,0.0,10.20,Credit Card,None,41.926811,-87.642605,POINT (-87.6426052473 41.9268111821),41.953400,-87.646007,POINT (-87.6460070664 41.9534000435)
1,37682f189f404780d57f266556003849c5fec752,a02e1f355041b6350e4d79a4de1e650b266f5dc1ea40d1...,2015-10-12 23:45:00+00:00,2015-10-12 23:45:00+00:00,300,1.0,NaN,NaN,11,15,...,0.0,5.45,Cash,None,41.978830,-87.771167,POINT (-87.771166703 41.9788295262),41.954028,-87.763399,POINT (-87.7633990316 41.9540276487)
2,7add88253c78688a6a0920cb30adbb4944418601,e637e1afe0dab0e2d04c635335602caf1045d43e3741c2...,2015-10-16 15:45:00+00:00,2015-10-16 15:45:00+00:00,360,0.8,1.703108e+10,1.703184e+10,8,32,...,1.0,6.25,Cash,None,41.909496,-87.630964,POINT (-87.630963601 41.9094956686),41.871016,-87.631407,POINT (-87.6314065252 41.8710158803)
3,7d597ed8fa15502482d3c98b74fdc084785c7f14,a538132e78f04101a86f83f4697f6944ea8c97acef8eb3...,2015-10-11 02:15:00+00:00,2015-10-11 02:15:00+00:00,780,6.6,1.703103e+10,1.703107e+10,77,7,...,1.0,16.65,Cash,None,41.994442,-87.665225,POINT (-87.6652247765 41.9944422482),41.921855,-87.646211,POINT (-87.6462109769 41.9218549112)
4,053c4b3fe66ca85a76998502de77bbc34ddf4668,0c1aa5b32422ba0cc2082cd87c91d95f8ec088421d886a...,2015-10-11 01:30:00+00:00,2015-10-11 01:45:00+00:00,480,2.0,1.703105e+10,1.703104e+10,5,4,...,0.0,7.45,Cash,None,41.950546,-87.676182,POINT (-87.6761824959 41.95054569650001),41.966806,-87.694020,POINT (-87.6940200973 41.9668063463)


After deciding whether you see any important issues, run the code cell below.

In [20]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
You can see the data by calling: 
```python
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=5).to_dataframe()
```

Some location fields have values of `None` or `NaN`. That is a problem if we want to use those fields.


### 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns:
- `year` - the year of the trips
- `num_trips` - the number of trips in that year

Hints:
- When using **GROUP BY** and **ORDER BY**, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The SQL code to **SELECT** the year from `trip_start_timestamp` is <code>SELECT EXTRACT(YEAR FROM trip_start_timestamp)</code>
- The **FROM** field can be a little tricky until you are used to it.  The format is:
    1. A backick (the symbol \`).
    2. The project name. In this case it is `bigquery-public-data`.
    3. A period.
    4. The dataset name. In this case, it is `chicago_taxi_trips`.
    5. A period.
    6. The table name. You used this as your answer in **1) Find the data**.
    7. A backtick (the symbol \`).

In [22]:
# Your code goes here
rides_per_year_query = """
                        select extract(year from trip_start_timestamp) as year, count(*) as num_trips
                        from `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        group by year
"""

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_year_result)

# Check your answer
q_3.check()

    year  num_trips
0   2013   27217300
1   2016   31756403
2   2017   24979611
3   2023    6495415
4   2018   20731105
5   2022    6382071
6   2021    3947677
7   2015   32385527
8   2014   37395079
9   2019   16476440
10  2020    3888831


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [23]:
#q_3.hint()
#q_3.solution()

### 4) Dive slightly deeper

You'd like to take a closer look at rides from 2016.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways:
1. Use a **WHERE** clause to limit the query to data from 2016.
2. Modify the query to extract the month rather than the year.

In [24]:
# Your code goes here
rides_per_month_query = """
                        select extract(month from trip_start_timestamp) as month, count(*) as num_trips
                        from `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                        where extract(year from trip_start_timestamp)=2016
                        group by month""" 

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_month_result)

# Check your answer
q_4.check()

    month  num_trips
0      10    2725340
1       2    2568433
2       8    2629482
3      12    2312992
4      11    2387790
5       3    2851106
6       9    2532650
7       5    2859147
8       6    2841872
9       1    2510389
10      4    2854290
11      7    2682912


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [25]:
#q_4.hint()
#q_4.solution()

### 5) Write the query

It's time to step up the sophistication of your queries.  Write a query that shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

Your results should have three columns:
- `hour_of_day` - sort by this column, which holds the result of extracting the hour from `trip_start_timestamp`.
- `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
- `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

Restrict your query to data meeting the following criteria:
- a `trip_start_timestamp` > **2016-01-01** and < **2016-04-01**
- `trip_seconds` > 0 and `trip_miles` > 0

You will use a common table expression (CTE) to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE -- instead you'll create those in the later **SELECT** statement below the CTE).

This is a much harder query than anything you've written so far.  Good luck!

In [27]:
# Your code goes here
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT trip_start_timestamp, trip_miles, trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp>'2016-01-01' and trip_start_timestamp<'2016-04-01' and trip_seconds>0 and trip_miles>0
               )
               SELECT extract(hour from trip_start_timestamp) as hour_of_day, count(*) as num_trips, 3600*sum(trip_miles)/sum(trip_seconds) as avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe() # Your code here

# View results
print(speeds_result)

# Check your answer
q_5.check()

    hour_of_day  num_trips    avg_mph
0             0     203092  20.191744
1             1     178046  18.628598
2             2     143447  18.444370
3             3     108899  19.273107
4             4      80067  27.599669
5             5      75786  33.065604
6             6     102254  28.533112
7             7     187585  19.884592
8             8     284223  16.787900
9             9     306854  18.434124
10           10     279762  20.091309
11           11     294006  20.926340
12           12     311522  20.063901
13           13     317225  19.766321
14           14     312629  19.309655
15           15     319953  18.515564
16           16     349455  17.168814
17           17     394324  14.641375
18           18     431991  15.381995
19           19     416743  17.795008
20           20     356279  20.347398
21           21     318363  22.584731
22           22     289886  21.129847
23           23     241690  20.259757


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the appropriate line below.

In [ ]:
#q_5.solution()

That's a hard query. If you made good progress towards the solution, congratulations!

# Keep going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in the Intro to SQL course.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-sql/discussion) to chat with other learners.*